In [ ]:
# Required Libraries
import pandas as pd
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from statsmodels.datasets import grunfeld
import statsmodels.api as sm
import statsmodels.formula.api as smf
from utils import read_data, plot_covariate_distributions, plot_match, compare_balance, sizeof_fmt, optimize_memory_df, plot_categorical_proportional_diff, compute_mean_differences_and_proportions, love_plot, sensitivity_analysis_k_neighbors
import spreg
import libpysal
import numpy as np

In [ ]:
data = read_data("../Data/processed/full_dataset_nonull.parquet.gzip")
data_final = data[data.gp.isin(['LCA & AOS','AOS only','LCA only'])]

In [ ]:
del data

### Filter out individuals that do not belong to the groups part of treatment and control

In [ ]:
df_treated = optimize_memory_df(data_final[data_final.treatment.isnull()==False])

### Filter out individuals that do not belong to the treatment and control for the whole 5 years

In [ ]:
# Get unique years
unique_years = set(df_treated['NOANNEE'])

# Group by patient_id and filter
df_treated_filtered = df_treated.groupby('uuid').filter(lambda x: set(x['NOANNEE']) == unique_years)

In [ ]:
df_treated_filtered.groupby(['NOANNEE','treatment']).uuid.nunique()

In [ ]:
del df_treated

In [ ]:
df_treated_filtered['year'] = df_treated_filtered['NOANNEE']-2016
df_multilevel_model = df_treated_filtered.set_index(['uuid','NOANNEE'])

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Load your data
# df = ...

# Define your variables
outcome_var = 'PRESTATIONS_BRUTES_AOS'  # The outcome variable name as a string
time_var = 'year'  # The time variable name as a string
covariates = ['cov1', 'cov2', 'cov3']  # List of covariate names as strings
group_var = 'patient_id'  # The grouping variable name as a string (e.g., patient ID)

# Convert time to string if it's not already, as Patsy requires this
df[time_var] = df[time_var].astype(str)

# Ordinary Least Squares (OLS) Model
ols_formula = f"{outcome_var} ~ {' + '.join(covariates + [time_var])}"
ols_model = smf.ols(ols_formula, data=df).fit()

# Mixed Effects Model with Random Intercepts
me_intercept_formula = f"{outcome_var} ~ {' + '.join(covariates + [time_var])}"
me_intercept_model = smf.mixedlm(me_intercept_formula, data=df, groups=df[group_var]).fit()

# Mixed Effects Model with Random Intercepts and Random Slopes
me_slope_formula = f"{outcome_var} ~ {' + '.join(covariates + [time_var])}"
me_slope_model = smf.mixedlm(me_slope_formula, data=df, 
                              groups=df[group_var], 
                              re_formula=f"~{time_var}").fit()

# Compare Models
models = [('OLS', ols_model), ('Mixed_Intercepts', me_intercept_model), ('Mixed_Slopes', me_slope_model)]
for name, model in models:
    print(f"{name} AIC: {model.aic}")

# Guide for Model Choice
# Lower AIC values suggest better-fitting models. However, also consider the complexity
# and interpretability of the model, as well as theoretical justifications for including
# random effects. Additionally, you may want to look at other measures like BIC, likelihood
# ratio tests, and cross-validation results when making your final decision.
